In [ ]:
import cv2
from ultralytics import YOLO
import pygame
import threading

# Initialize pygame mixer for sound
pygame.mixer.init()

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")  # Use YOLOv8 with pre-trained weights

# Define a danger zone (coordinates of the top-left and bottom-right corners)
danger_zone = [(100, 100), (400, 400)]  # Example: (top-left, bottom-right)

# Define the path to the alert sound file
alert_sound = "/media/enma/Torch/cv/cv_detection_project_0/beep-warning.mp3"  # Replace with the correct path to your sound file

# Load the sound
alert_sound_obj = pygame.mixer.Sound(alert_sound)

# Define the class of interest (person only for triggering alerts)
class_of_interest = "person"

# Function to check if a detected object intersects with the danger zone
def is_in_danger_zone(box, zone):
    x1, y1, x2, y2 = box  # Object's bounding box coordinates
    (zone_x1, zone_y1), (zone_x2, zone_y2) = zone  # Danger zone coordinates
    
    # Check if any part of the object is inside the danger zone (partial overlap)
    overlap_x = (x1 < zone_x2) and (x2 > zone_x1)  # Horizontal overlap
    overlap_y = (y1 < zone_y2) and (y2 > zone_y1)  # Vertical overlap
    return overlap_x and overlap_y  # True if any part of the person overlaps with the danger zone

# Function to play the alert sound continuously
def play_alert_sound():
    while True:
        if not pygame.mixer.get_busy():  # Check if the sound is not playing
            alert_sound_obj.play()
        pygame.time.wait(100)  # Wait for a short period before checking again

# Initialize webcam video capture
cap = cv2.VideoCapture(0)  # 0 for the default webcam

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Initialize a flag to control sound alerts
sound_thread = threading.Thread(target=play_alert_sound)
sound_thread.daemon = True  # Ensure the thread will exit when the main program exits
sound_thread.start()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Convert frame to grayscale to simulate CCTV footage
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_frame_colored = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)  # Keep 3 channels for consistency

    # Simulate an infrared effect using a color map
    infrared_frame = cv2.applyColorMap(gray_frame, cv2.COLORMAP_JET)

    # Run YOLOv8 model prediction on the current frame
    results = model(frame)

    # Draw the danger zone in red on the frame
    cv2.rectangle(frame, danger_zone[0], danger_zone[1], (0, 0, 255), 2)  # Draw the danger zone in red
    cv2.rectangle(gray_frame_colored, danger_zone[0], danger_zone[1], (0, 0, 255), 2)  # On grayscale
    cv2.rectangle(infrared_frame, danger_zone[0], danger_zone[1], (0, 0, 255), 2)  # On infrared

    # Process results
    alert_triggered = False  # Flag to track if alert should be raised
    for result in results:
        for i, box in enumerate(result.boxes.xyxy.tolist()):  # Extract box coordinates
            x1, y1, x2, y2 = map(int, box)

            # Correctly convert tensor to an integer index
            class_index = int(result.boxes.cls[i])
            class_name = result.names[class_index]

            # Check for the person class specifically
            if class_name == class_of_interest:
                # Draw the bounding box on all feeds
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.rectangle(gray_frame_colored, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.rectangle(infrared_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

                # Draw the label on all feeds
                confidence = result.boxes.conf[i].item()
                label = f"{class_name}: {confidence:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                cv2.putText(gray_frame_colored, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                cv2.putText(infrared_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                # Check if any part of the detected person intersects with the danger zone
                if is_in_danger_zone((x1, y1, x2, y2), danger_zone):
                    alert_triggered = True
                    cv2.putText(frame, "ALERT! Person in Danger Zone", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    cv2.putText(gray_frame_colored, "ALERT! Person in Danger Zone", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    cv2.putText(infrared_frame, "ALERT! Person in Danger Zone", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Play the alert sound if a person is in the danger zone and sound is not already playing
    if alert_triggered:
        if not pygame.mixer.get_busy():  # If the sound is not playing, play it
            alert_sound_obj.play(loops=-1)  # Play sound indefinitely
    else:
        if pygame.mixer.get_busy():  # If no persons are in the danger zone, stop the sound
            alert_sound_obj.stop()

    # Display the resulting frames
    cv2.imshow('YOLOv8 Zone Detection (Color)', frame)
    cv2.imshow('YOLOv8 Zone Detection (Grayscale)', gray_frame_colored)
    cv2.imshow('YOLOv8 Zone Detection (Infrared)', infrared_frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()  # Quit the mixer when done
